In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy
from itertools import repeat

### Open gadm0_precalculated
Geojson downloaded from AGOL

In [5]:
# gadm_precalc = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/gadm0_precalculated_20220224_backup2.geojson')

gadm_nrc = gpd.read_file('/Users/sofia/Documents/HE_Data/NRC/NRC_2022/NRC_final_20220623.geojson')

In [6]:
gadm_nrc.head()

,GID_0,NAME_0,Area_Country,x,y,jpg_url,has_priority,has_raisg,GlobalID,max_highlited_sp,...,mammals_mar,fishes_mar,endemic_mammals_mar,endemic_fishes_mar,nspecies_mar,total_endemic_mar,Area_EEZ,Global_SPI_mar,filter_similar_mar,geometry
0,ABW,Aruba,181.94,-69.97,12.51,https://live.staticflickr.com/1952/31416683438...,1.0,0.0,fe9f6eb0-f4f8-4f29-875a-5cbb3219e4e5,4.0,...,30,1648,0,0,1678,0,29970.30,23.40,"{'filter_Area_Country': ['CUW', 'GRD', 'LVA', ...",POINT (-69.97000 12.51000)
1,AFG,Afghanistan,643857.48,66.03,33.83,https://p1.pxfuel.com/preview/967/12/53/afghan...,1.0,0.0,193ba976-0e5a-4cf6-9b09-d00bf83f4557,5.0,...,0,0,0,0,0,0,NaN,nan,None,POINT (66.03000 33.83000)
2,AGO,Angola,1247421.58,17.58,-12.34,https://live.staticflickr.com/3787/13698381215...,1.0,0.0,174ce788-4f67-4ae0-922f-d2ddac87f8c3,24.0,...,46,1372,0,2,1418,2,495859.76,23.40,"{'filter_Area_Country': ['NFK', 'BMU', 'CCK', ...",POINT (17.58000 -12.34000)
3,AIA,Anguilla,83.30,-63.05,18.21,https://live.staticflickr.com/8063/8194570372_...,1.0,0.0,9f5f24d8-8b21-49a8-8f55-90b47cf63e7b,2.0,...,25,1508,0,0,1533,0,90157.96,23.40,"{'filter_Area_Country': ['ERI', 'AZE', 'FIN', ...",POINT (-63.05000 18.21000)
4,ALA,Åland,1506.26,19.97,60.24,https://p1.pxfuel.com/preview/294/670/561/alan...,1.0,0.0,2b45351b-a335-490e-914e-7748d4f41f66,1.0,...,0,0,0,0,0,0,NaN,nan,None,POINT (19.97000 60.24000)


In [7]:
len(gadm_nrc)

254

In [8]:
type(gadm_nrc)

geopandas.geodataframe.GeoDataFrame

### Open gadm version 4.0

In [9]:
gadm40 = gpd.read_file('/Users/sofia/Documents/HE_Data/gadm/gadm404-shp/gadm404.shp')


In [10]:
len(gadm40)

348904

### Create new table with gadm40 in which we only have GID_0 and NAME_0

In [11]:
gadm40_GID = gadm40[['GID_0', 'NAME_0']]
gadm40_GID = gadm40_GID.groupby('GID_0')
gadm40_GID = gadm40_GID.first()
gadm40_GID = gadm40_GID.reset_index()
gadm40_GID

,GID_0,NAME_0
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland
...,...,...
258,Z09,Sang
259,ZAF,South Africa
260,ZMB,Zambia
261,ZNC,Northern Cyprus


In [12]:
gadm40_GID = gadm40_GID.rename(columns={'GID_0':'GID', 'NAME_0':'NAME'})
gadm40_GID

,GID,NAME
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland
...,...,...
258,Z09,Sang
259,ZAF,South Africa
260,ZMB,Zambia
261,ZNC,Northern Cyprus


In [13]:
gadm = pd.merge(gadm_nrc, gadm40_GID, how='left', left_on='GID_0', right_on='GID')
gadm.head()

,GID_0,NAME_0,Area_Country,x,y,jpg_url,has_priority,has_raisg,GlobalID,max_highlited_sp,...,endemic_mammals_mar,endemic_fishes_mar,nspecies_mar,total_endemic_mar,Area_EEZ,Global_SPI_mar,filter_similar_mar,geometry,GID,NAME
0,ABW,Aruba,181.94,-69.97,12.51,https://live.staticflickr.com/1952/31416683438...,1.0,0.0,fe9f6eb0-f4f8-4f29-875a-5cbb3219e4e5,4.0,...,0,0,1678,0,29970.30,23.40,"{'filter_Area_Country': ['CUW', 'GRD', 'LVA', ...",POINT (-69.97000 12.51000),ABW,Aruba
1,AFG,Afghanistan,643857.48,66.03,33.83,https://p1.pxfuel.com/preview/967/12/53/afghan...,1.0,0.0,193ba976-0e5a-4cf6-9b09-d00bf83f4557,5.0,...,0,0,0,0,NaN,nan,None,POINT (66.03000 33.83000),AFG,Afghanistan
2,AGO,Angola,1247421.58,17.58,-12.34,https://live.staticflickr.com/3787/13698381215...,1.0,0.0,174ce788-4f67-4ae0-922f-d2ddac87f8c3,24.0,...,0,2,1418,2,495859.76,23.40,"{'filter_Area_Country': ['NFK', 'BMU', 'CCK', ...",POINT (17.58000 -12.34000),AGO,Angola
3,AIA,Anguilla,83.30,-63.05,18.21,https://live.staticflickr.com/8063/8194570372_...,1.0,0.0,9f5f24d8-8b21-49a8-8f55-90b47cf63e7b,2.0,...,0,0,1533,0,90157.96,23.40,"{'filter_Area_Country': ['ERI', 'AZE', 'FIN', ...",POINT (-63.05000 18.21000),AIA,Anguilla
4,ALA,Åland,1506.26,19.97,60.24,https://p1.pxfuel.com/preview/294/670/561/alan...,1.0,0.0,2b45351b-a335-490e-914e-7748d4f41f66,1.0,...,0,0,0,0,NaN,nan,None,POINT (19.97000 60.24000),ALA,Åland


In [16]:
gadm[['GID_0', 'NAME_0', 'GID', 'NAME']]

,GID_0,NAME_0,GID,NAME
0,ABW,Aruba,ABW,Aruba
1,AFG,Afghanistan,AFG,Afghanistan
2,AGO,Angola,AGO,Angola
3,AIA,Anguilla,AIA,Anguilla
4,ALA,Åland,ALA,Åland
...,...,...,...,...
249,TZA,Tanzania,TZA,Tanzania
250,UGA,Uganda,UGA,Uganda
251,UKR,Ukraine,UKR,Ukraine
252,UMI,United States Minor Outlying Islands,UMI,United States Minor Outlying Islands


In [17]:
gadm2 = gadm[gadm.NAME_0!=gadm.NAME]
gadm2[['NAME_0', 'NAME']]

,NAME_0,NAME
39,Northern Cyprus,NaN
67,Republic of Congo,Republic of the Congo
71,Cape Verde,Cabo Verde
78,Czech Republic,Czechia
116,Hong Kong,NaN
155,Macao,NaN
164,Macedonia,North Macedonia
204,Palestina,Palestine
207,Reunion,Réunion
216,Saint Helena,"Saint Helena, Ascension and Tristan da Cunha"


In [18]:
gadm.columns

Index(['GID_0', 'NAME_0', 'Area_Country', 'x', 'y', 'jpg_url', 'has_priority',
       'has_raisg', 'GlobalID', 'max_highlited_sp', 'continent', 'GNI_PPP',
       'sentence', 'hm_ter', 'hm_no_ter', 'hm_vh_ter', 'Global_SPI_ter',
       'Pop2020', 'SPI_ter', 'prop_protected_ter', 'protection_needed_ter',
       'amphibians', 'birds', 'mammals', 'reptiles', 'endemic_amphibians',
       'endemic_birds', 'endemic_mammals', 'endemic_reptiles', 'nspecies_ter',
       'total_endemic_ter', 'filter_similar_ter', 'Marine', 'Pop2020_EEZ',
       'hm_no_mar', 'hm_mar', 'hm_vh_mar', 'SPI_mar', 'protection_needed_mar',
       'prop_protected_mar', 'mammals_mar', 'fishes_mar',
       'endemic_mammals_mar', 'endemic_fishes_mar', 'nspecies_mar',
       'total_endemic_mar', 'Area_EEZ', 'Global_SPI_mar', 'filter_similar_mar',
       'geometry', 'GID', 'NAME'],
      dtype='object')

In [19]:
gadm.NAME.fillna(gadm.NAME_0, inplace=True)


In [21]:
gadm2 = gadm[gadm.NAME_0!=gadm.NAME]
gadm2[['NAME_0', 'NAME']]

,NAME_0,NAME
67,Republic of Congo,Republic of the Congo
71,Cape Verde,Cabo Verde
78,Czech Republic,Czechia
164,Macedonia,North Macedonia
204,Palestina,Palestine
207,Reunion,Réunion
216,Saint Helena,"Saint Helena, Ascension and Tristan da Cunha"


In [22]:
gadm.NAME_0 = gadm.NAME
gadm = gadm.drop(columns={'NAME', 'GID'})

In [23]:
gadm.NAME_0[gadm.GID_0=='CZE']

78    Czechia
Name: NAME_0, dtype: object

In [25]:
gadm_nrc.NAME_0[gadm_nrc.GID_0=='CZE']

78    Czech Republic
Name: NAME_0, dtype: object

In [26]:
gadm.NAME_0[gadm.GID_0=='HKG']

116    Hong Kong
Name: NAME_0, dtype: object

In [27]:
gadm[gadm.NAME_0 == 'Swaziland']

,GID_0,NAME_0,Area_Country,x,y,jpg_url,has_priority,has_raisg,GlobalID,max_highlited_sp,...,mammals_mar,fishes_mar,endemic_mammals_mar,endemic_fishes_mar,nspecies_mar,total_endemic_mar,Area_EEZ,Global_SPI_mar,filter_similar_mar,geometry
231,SWZ,Swaziland,17383.02,31.5,-26.57,https://upload.wikimedia.org/wikipedia/commons...,1.0,0.0,263bef95-4866-4ca7-b0e2-b63d30d3fffb,1.0,...,0,0,0,0,0,0,NaN,nan,None,POINT (31.50000 -26.57000)


In [28]:
len(gadm)

254

In [29]:
gadm.columns

Index(['GID_0', 'NAME_0', 'Area_Country', 'x', 'y', 'jpg_url', 'has_priority',
       'has_raisg', 'GlobalID', 'max_highlited_sp', 'continent', 'GNI_PPP',
       'sentence', 'hm_ter', 'hm_no_ter', 'hm_vh_ter', 'Global_SPI_ter',
       'Pop2020', 'SPI_ter', 'prop_protected_ter', 'protection_needed_ter',
       'amphibians', 'birds', 'mammals', 'reptiles', 'endemic_amphibians',
       'endemic_birds', 'endemic_mammals', 'endemic_reptiles', 'nspecies_ter',
       'total_endemic_ter', 'filter_similar_ter', 'Marine', 'Pop2020_EEZ',
       'hm_no_mar', 'hm_mar', 'hm_vh_mar', 'SPI_mar', 'protection_needed_mar',
       'prop_protected_mar', 'mammals_mar', 'fishes_mar',
       'endemic_mammals_mar', 'endemic_fishes_mar', 'nspecies_mar',
       'total_endemic_mar', 'Area_EEZ', 'Global_SPI_mar', 'filter_similar_mar',
       'geometry'],
      dtype='object')

In [30]:
gadm.to_file('/Users/sofia/Documents/HE_Data/NRC/NRC_2022/NRC_final_20220623_gadm4.geojson',driver='GeoJSON')